<a href="https://colab.research.google.com/github/wolffg777/descriptors_to_music_rec/blob/main/MusicRec01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO:
WORD2VEC KERAS CLASSIFICATION PROBLEM

- word2vec on descriptors and genres
  - first just working
  - then working out OOV (out of vocabulary) problems #fastText maybe ask chatgpt
- testing it to see improvement
- implementing dates (as continuous)
- easier things first
- spotify data integration


-brainstorm an idea for the name of the independent study on askbanner (limited to 30 characters)



In [1]:
from google.colab import files
files.upload()
!pip install kaggle
!chmod 600 /root/.kaggle/kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!kaggle datasets download -d tobennao/rym-top-5000
!unzip rym-top-5000.zip
!pip install spotipy

Saving kaggle.json to kaggle.json
rym-top-5000.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  rym-top-5000.zip
replace rym_clean1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace rym_raw1.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [106]:
import pandas as pd
df = pd.read_csv('rym_clean1.csv')
# df.head(1)

# Remove rating_count, review_count, and avg_rating columns
df = df.drop(columns=['avg_rating'])
df = df.drop(columns=['rating_count'])
df = df.drop(columns=['review_count'])
df = df.drop(columns=['Unnamed: 0'])
df = df.drop(columns=['position'])
df = df.drop(columns=['release_type'])
df.head(5)

,release_name,artist_name,release_date,primary_genres,secondary_genres,descriptors
0,OK Computer,Radiohead,1997-06-16,"Alternative Rock, Art Rock",NaN,"melancholic, anxious, futuristic, malevocals, ..."
1,Kid A,Radiohead,2000-10-03,"Art Rock, Experimental Rock, Electronic","Ambient, Electronic, IDM","cold, melancholic, futuristic, anxious, atmosp..."
2,The Dark Side of the Moon,Pink Floyd,1973-03-23,"Art Rock, Progressive Rock","Psychedelic Rock, Space Rock","philosophical, atmospheric, introspective, exi..."
3,Loveless,My Bloody Valentine,1991-11-11,"Shoegaze, Noise Pop","Dream Pop, Neo-Psychedelia","noisy, ethereal, atmospheric, romantic, love, ..."
4,My Beautiful Dark Twisted Fantasy,Kanye West,2010-11-22,"Pop Rap, Hip Hop",Art Pop,"epic, boastful, passionate, sampling, hedonist..."


In [107]:
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')

In [108]:
df['descriptors'].head()

0    melancholic, anxious, futuristic, malevocals, ...
1    cold, melancholic, futuristic, anxious, atmosp...
2    philosophical, atmospheric, introspective, exi...
3    noisy, ethereal, atmospheric, romantic, love, ...
4    epic, boastful, passionate, sampling, hedonist...
Name: descriptors, dtype: object

In [109]:
#step 1: tokenize the descriptors
desc_words = df['descriptors'].str.split(', ')
df['descriptors'] = desc_words
# desc_encoded = df['descriptors'].str.join('|').str.get_dummies()

In [110]:
print(desc_words.iloc[0])

['melancholic', 'anxious', 'futuristic', 'malevocals', 'existential', 'alienation', 'atmospheric', 'lonely', 'cold', 'pessimistic']


In [111]:
print(desc_words)

0       [melancholic, anxious, futuristic, malevocals,...
1       [cold, melancholic, futuristic, anxious, atmos...
2       [philosophical, atmospheric, introspective, ex...
3       [noisy, ethereal, atmospheric, romantic, love,...
4       [epic, boastful, passionate, sampling, hedonis...
                              ...                        
4995    [instrumental, atmospheric, bittersweet, senti...
4996    [fantasy, epic, triumphant, melodic, atmospher...
4997    [futuristic, ominous, sciencefiction, atmosphe...
4998    [instrumental, ritualistic, hypnotic, atmosphe...
4999    [femalevocals, melodic, playful, uplifting, ro...
Name: descriptors, Length: 5000, dtype: object


In [112]:
# Word2Vec for descriptors
word2vec_model = Word2Vec(desc_words, min_count=5)
vocab = word2vec_model.wv.index_to_key  # list of words known to the model
vocab_size = len(vocab)

In [113]:
print([word2vec_model.wv["melancholic"]])
print(df['descriptors'].head())

[array([-0.14469866,  0.15786278,  0.0718255 , -0.17790589,  0.0704151 ,
       -0.35739797,  0.56255263,  0.43680507, -0.39365613, -0.39470676,
        0.08105635, -0.55804664,  0.08440865, -0.05598122, -0.12093981,
       -0.34149826,  0.15418002, -0.32056704,  0.02019256, -0.7093065 ,
        0.12183452, -0.06735524, -0.05843396,  0.03217513, -0.34297204,
       -0.08172467, -0.0025688 , -0.13263457, -0.15534109, -0.17710522,
       -0.03154664,  0.04773464,  0.03325362, -0.10140605, -0.13129716,
        0.3563522 ,  0.4144083 , -0.32050014, -0.11823205,  0.06533086,
        0.3144787 , -0.38678038, -0.04588588,  0.04610044,  0.19633225,
       -0.05023242, -0.04106596,  0.08354156,  0.09961169,  0.15198286,
        0.0672507 , -0.20532519,  0.21245088,  0.21322621, -0.12183204,
        0.15750527,  0.11609888,  0.28057155, -0.0544243 ,  0.20627406,
        0.33652517, -0.15809079,  0.18109237, -0.19838649,  0.07808407,
        0.12705564, -0.16316915,  0.0822459 , -0.18673724, -0.0

In [114]:
def sentence_to_vec(sentence):
    vectors = [word2vec_model.wv[word] for word in sentence if word in vocab]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)  # Return a zero vector if no words are in the vocab

df['descriptor_avg_vec'] = df['descriptors'].apply(sentence_to_vec)

# Pad these averaged descriptor vectors
padded_descriptors = pad_sequences(df['descriptor_avg_vec'].tolist(), maxlen=10, dtype='float32')

In [115]:
print(df['descriptor_avg_vec'])

0       [0.003869877, 0.12148881, -0.06330204, 0.01615...
1       [0.043525595, 0.10464637, -0.08737961, 0.01534...
2       [0.04179308, 0.11737891, -0.08742379, -0.01027...
3       [0.016385501, 0.15327671, -0.035801798, -0.105...
4       [-0.016238766, 0.20007169, -0.0808249, 0.03716...
                              ...                        
4995    [-0.07260514, 0.19884737, 0.042687867, -0.2286...
4996    [0.06662184, 0.13272227, -0.10998907, 0.021949...
4997    [0.13394007, 0.12176348, -0.16629636, 0.087144...
4998    [0.09844835, 0.1186883, -0.10839287, -0.045081...
4999    [-0.06669199, 0.19716224, 0.011737647, -0.1135...
Name: descriptor_avg_vec, Length: 5000, dtype: object


In [117]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [118]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['release_name_label'] = encoder.fit_transform(df['release_name'])

# Prepare training data
X = np.array(padded_descriptors)
y = df['release_name_label'].values

In [119]:
print(X.shape)
print(y.shape)

(5000, 10)
(5000,)


In [124]:
# Build the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length),
    LSTM(128),
    Dense(len(df['release_name'].unique()), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=64)

Epoch 1/10
79/79 [==============================] - 6s 42ms/step - loss: 8.4829 - accuracy: 0.0000e+00
Epoch 2/10
79/79 [==============================] - 4s 49ms/step - loss: 8.5477 - accuracy: 2.0000e-04
Epoch 3/10
79/79 [==============================] - 4s 54ms/step - loss: 8.5086 - accuracy: 4.0000e-04
Epoch 4/10
79/79 [==============================] - 5s 66ms/step - loss: 8.5111 - accuracy: 0.0000e+00
Epoch 5/10
79/79 [==============================] - 8s 103ms/step - loss: 8.5144 - accuracy: 6.0000e-04
Epoch 6/10
79/79 [==============================] - 6s 79ms/step - loss: 8.5140 - accuracy: 2.0000e-04
Epoch 7/10
79/79 [==============================] - 7s 89ms/step - loss: 8.5091 - accuracy: 8.0000e-04
Epoch 8/10
79/79 [==============================] - 5s 67ms/step - loss: 8.5084 - accuracy: 0.0010
Epoch 9/10
79/79 [==============================] - 4s 48ms/step - loss: 8.5082 - accuracy: 8.0000e-04
Epoch 10/10
79/79 [==============================] - 4s 54ms/step - loss: 8.

things to do: stricter filtering of descriptors
one-hot the genres, word2vec (number of dimensions depends on embedding model, basic word2vec is 30 - 100), the descriptors and average the value and put it in space. tutorials on youtube seem good.

try to find a taxonomy of genres online, if not split up words and vaguely separating the musical words

Partial credit accuracy/reward system where "radiohead" for atmospheric even if the album recommended is not atmospheric

In [ ]:
import os
import base64
from requests import post, get
import json
import spotipy
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials


os.environ['SPOTIPY_CLIENT_ID'] = '08a0640020ff4e5d9713aaeacfbd1886'
os.environ['SPOTIPY_CLIENT_SECRET'] = '81cefe24f94b4f24af9fe1ad2e70ef4a'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://example.com/callback'

#Authorization Code Flow
scopes = "user-library-read user-top-read user-follow-read"

sp = spotipy.Spotify(
        auth_manager=spotipy.SpotifyOAuth(
          scope=scopes, open_browser=False))

In [ ]:
#Get Specified User's Playlists
playlists = sp.user_playlists('wolffg7')

while playlists:
    for i, playlist in enumerate(playlists['items']):
        #print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
        print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))

    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None


Go to the following URL: https://accounts.spotify.com/authorize?client_id=08a0640020ff4e5d9713aaeacfbd1886&response_type=code&redirect_uri=http%3A%2F%2Fexample.com%2Fcallback&scope=user-library-read+user-top-read+user-follow-read
Enter the URL you were redirected to: http://example.com/callback?code=AQA4IC9iTbOmsTlsroqvltP65Ztyqs8e5E4W6GHhHQeDNJXCtyMWXohPySl6iXMunJTz-3mHRpq4u4k9__bjznHVTPPmnDpH9eDvv6UjnxiogtTFw9ARKmE0p-qGPNbz_QDUqKh8fDx7Aql6XEnztErqCqCk0Nx4Do72YC9QzVmbq0-DikM9k9yABE61aKyx65HDZjCJbv6xvUvBN2od9Dvnf43rvut4sE1-QuR0hJePgbI
   1 My playlist #1,813
   2 SEGA SUNSET
   3 Ready For The Floor
   4 Trombone Concerto: Movement I - Studio
   5 Who Can I Run To
   6 Rich Baby Daddy (feat. Sexyy Red & SZA)
   7 War
   8 I Miss You (feat. Teddy Pendergrass)
   9 Archangel
  10 Halcyon and On and On
  11 Truth Is Light
  12 Destiny
  13 Bright Lights
  14 Ya Just Need 2 Believe in Yaself
  15 Final Credits - Radio Edit
  16 House in the Hills
  17 Burnt Orange - Edit
  18 Rose & Beast 

In [ ]:
album_name = "OK Computer"
artist_name = "Radiohead"

# Search for the album
results = sp.search(q=f"album:{album_name} artist:{artist_name}", type="album")

album_id = results['albums']['items'][0]['id']

album_tracks = sp.album_tracks(album_id)

for track in album_tracks['items']:
    print(track['name'])


Airbag
Paranoid Android
Subterranean Homesick Alien
Exit Music (For A Film)
Let Down
Karma Police
Fitter Happier
Electioneering
Climbing Up the Walls
No Surprises
Lucky
The Tourist


model architecture: pytorch simple neural network (feedforward neural network).

get a model working before break, even if its simpler (in terms of model architecture and pre-processing).

in recommendation stage, top n choices of different artists because otherwise it will just recommend songs from the same album
|
also figure out a way to vectorize/umbrella categories for genres
|
add column for frequency of each genre like
col 1 : space rock
col 2 : rock
col 3 : frequency 3
|
also separate the words in the descriptors (ie "malevocals" should be "male vocals")

still do this, but only useful for bringing in new information that is song specific, like lyrics and popularity

In [ ]:
# CODE GRAVEYARD

# # Flatten the lists in the "descriptors" column
# flattened_descriptors = df['descriptors'].explode()

# # Count the number of unique strings
# unique_count = flattened_descriptors.nunique()

# flattened_descriptors.head(5)

# # Use value_counts to count the frequency of elements in the column
# element_counts = flattened_descriptors.value_counts()

# # Print the result
# print(element_counts)



# # One-hot-encoding for DESCRIPTORS

# # Turn descriptors into a list of strings
# df['descriptors'] = df['descriptors'].str.split(',')

# descriptors_encoded = df['descriptors'].str.join('|').str.get_dummies()

# descriptor_frequencies = descriptors_encoded.iloc[:, 2:].sum()

# # print (descriptor_frequencies)

# rarity_threshold = 10

# # Clean column names by stripping spaces
# descriptors_encoded.columns = descriptors_encoded.columns.str.strip()

# rare_descriptors = descriptor_frequencies[descriptor_frequencies < rarity_threshold]

# # print(rare_descriptors)
# # print(len(rare_descriptors))